### Import Modules

In [ ]:
# Preprocess import
import pandas as pd
import numpy as np

# pytorch import
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

### Run and test the simple 2-layer lstm predictor

Before that, go to `/utils` and run `preparation.py` then `traj_token.py`, make sure your data is located like `/data/taxi/2018-10-1.csv`, ...

In [ ]:
# Specify the file path
file_path = "../data/token_traj.csv"  # Replace with your actual file path

# Load the CSV file
data = pd.read_csv(file_path)

# Display the first five rows
print(data.head(5))

# Get the number of rows
num_rows = len(data)

print(f"The dataset contains {num_rows} rows.")

### Load, Split and Train
Remember set sample = 0 if you want to use the whole dataset. The whole dataset takes about 45 minutes to load in.

In [ ]:
class TrajectoryDataset(Dataset):
    def __init__(self, data_path, sample=1, max_rows=50000):
        if sample:
            self.data = pd.read_csv(data_path, nrows=max_rows)
        else:
            self.data = pd.read_csv(data_path)
        self.x = self.data['processed_trajectory'].apply(eval).tolist()
        self.y = self.data['time_elapsed'].values.astype(np.float32)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.x[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.float32)
        return x, y

In [ ]:
data_path = "../data/token_traj.csv"
dataset = TrajectoryDataset(data_path, sample = 0)

# Split the dataset into train/val/test = 7/1/2
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [ ]:
def mape_loss(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true))

In [ ]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, device="cpu", batch_size=32):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        train_mape = 0
        train_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}")
        cnt = 0
        for x_batch, y_batch in train_bar:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            predictions = model(x_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += (loss.item() / batch_size)
            train_mape += mape_loss(y_batch, predictions).item()
            cnt += 1

            # print("Y true: ", y_batch[0])
            # print("Y pred: ", predictions[0])
            train_bar.set_postfix({"Train Loss": (train_loss / cnt), "Train RMSE": (np.sqrt(train_loss / cnt)), "Train MAPE": (train_mape / cnt)})
        
        # print(f"Epoch {epoch+1}, Train MSE Loss: {train_loss / cnt}, Train RMSE: {np.sqrt(train_loss / cnt)}, Train MAPE: {(train_mape / cnt)}")

        # Validation phase
        model.eval()
        val_loss = 0
        val_mape = 0
        val_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{epochs}")
        cnt_val = 0
        with torch.no_grad():
            for x_batch, y_batch in val_bar:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                predictions = model(x_batch)
                loss = criterion(predictions, y_batch)
                val_loss += loss.item() / batch_size
                val_mape += mape_loss(y_batch, predictions).item()
                cnt_val += 1
                val_bar.set_postfix({"Val Loss": (val_loss / cnt_val), "Val RMSE": (np.sqrt(val_loss / cnt_val)), "Val MAPE": (val_mape / cnt_val)})
        
        # print(f"Epoch {epoch+1}, Val Loss: {val_loss / cnt_val}, Val MAPE: {np.sqrt(val_loss / cnt_val)}, Train MAPE: {(val_mape / cnt_val)}")

In [ ]:
def test_model(model, test_loader, device="cpu", batch_size = 32):
    model = model.to(device)
    model.eval()
    test_loss = 0
    test_mape = 0
    criterion = nn.MSELoss()
    test_bar = tqdm(test_loader, desc="Testing")
    cnt = 0
    with torch.no_grad():
        for x_batch, y_batch in test_bar:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            predictions = model(x_batch)
            loss = criterion(predictions, y_batch)
            test_loss += loss.item() / batch_size
            test_mape += mape_loss(y_batch, predictions).item()
            cnt += 1
            test_bar.set_postfix({"Test Loss": (test_loss / cnt), "Test RMSE": (np.sqrt(test_loss / cnt)), "Test MAPE": (test_mape / cnt)})

### Bi-LSTM

In [ ]:
# Define the bi-LSTM model
class BiLSTMTimePredictor(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=128, num_layers=2):
        super(BiLSTMTimePredictor, self).__init__()
        self.encoder = nn.LSTM(
            input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True
        )
        self.classifier = nn.Linear(hidden_dim * 2 * num_layers, 1)  # *2 because bidirectional
        
    def forward(self, x):
        # print("DEBUG: x shape: ", x.shape)
        _, (hidden, _) = self.encoder(x)  # hidden shape: (num_layers*2, batch_size, hidden_dim)
        # print("DEBUG: hidden shape before: ", hidden.shape)
        hidden = hidden.permute(1, 0, 2).reshape(x.size(0), -1)  # Flatten hidden states
        # print("DEBUG: hidden shape: ", hidden.shape)
        return self.classifier(hidden).squeeze(-1)

This takes 73 minutes on the 4090 gpu server. Final RMSE=104, MAPE=0.514

In [ ]:
# Train the model on both CPU and GPU
model = BiLSTMTimePredictor(num_layers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(model, train_loader, val_loader, epochs=10, lr=0.0005, device=device)

In [ ]:
test_model(model, test_loader, device=device)

### Transformer Encoder

In [ ]:
class TransformerTimePredictor(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=128, nhead=4, num_encoder_layers=4, mlp_hidden_dim=256):
        """
        Transformer-based model for time prediction.

        Args:
            input_dim (int): Number of features in each input step (e.g., 2 for [x, y]).
            hidden_dim (int): Embedding dimension for the Transformer.
            nhead (int): Number of attention heads in the Transformer.
            num_encoder_layers (int): Number of Transformer encoder layers.
            mlp_hidden_dim (int): Number of hidden units in the MLP classifier.
        """
        super(TransformerTimePredictor, self).__init__()

        # Input embedding layer
        self.embedding = nn.Linear(input_dim, hidden_dim)

        # Transformer encoder
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=nhead, dim_feedforward=hidden_dim * 4, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)

        # MLP classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, mlp_hidden_dim),
            nn.ReLU(),
            nn.Linear(mlp_hidden_dim, mlp_hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(mlp_hidden_dim // 2, 1),  # Output a single value
        )

    def forward(self, x):
        # Embed input
        # print("X input: ", x[0])
        x = self.embedding(x)  # Shape: (batch_size, seq_length, hidden_dim)
        
        # Transformer encoding
        x = self.transformer_encoder(x)  # Shape: (batch_size, seq_length, hidden_dim)
        
        # Use only the first token's representation for classification
        x = x[:, 0, :]  # Shape: (batch_size, hidden_dim)
        
        # Classifier
        output = self.classifier(x)  # Shape: (batch_size, 1)
        # print(output)
        return output.squeeze(-1)

In [ ]:
model = TransformerTimePredictor(input_dim=3, hidden_dim=128, nhead=4, num_encoder_layers=4, mlp_hidden_dim=256)

train_model(model, train_loader, val_loader, epochs=5, device=device)
test_model(model, test_loader, device=device)